In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
# GOING BACK TO RIGHT DIRECTORY
%pwd

'e:\\Projects\\E2E Emotion Detection from text\\Emotion-Detection-using-ML'

In [8]:
import pandas as pd

In [35]:
data = pd.read_parquet('E:/Projects/E2E Emotion Detection from text/Emotion-Detection-using-ML/artifacts/data_ingestion/train-00000-of-00001.parquet',
                       engine='pyarrow')

In [37]:
data.sample(5)

,text,label
246842,i wanted to cry and dream hurtful things and h...,0
174315,i have been feeling better,1
293560,i feel like such a completely heartless asshol...,3
383201,i feel amazed or be in awe at what a miracle h...,5
284493,i have to admit i was feeling shall we say tender,2


In [38]:
data.isna().sum()

text     0
label    0
dtype: int64

In [39]:
data.shape

(416809, 2)

In [40]:
# Label wise distribution in percentage
data["label"].value_counts()*100/data["label"].sum()

label
1    21.775177
0    18.706490
3     8.847483
4     7.364850
2     5.333774
5     2.311086
Name: count, dtype: float64

In [41]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [47]:
from src.ML_emotion_detection.constants import *
from src.ML_emotion_detection.utils.common import read_yaml, create_directories

In [43]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [44]:
from src.ML_emotion_detection import logger

In [45]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_parquet(self.config.unzip_data_dir,engine="pyarrow")
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [54]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-11-17 21:46:21,673: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-17 21:46:21,679: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-17 21:46:21,687: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-17 21:46:21,691: INFO: common: created directory at: artifacts]
[2024-11-17 21:46:21,694: INFO: common: created directory at: artifacts/data_validation]
